# Why use custom tools?

- Enhanced productivity: Automate repetitive tasks and streamline workflows.
- Improved accuracy: Provide precise and consistent outputs, reducing human error.
- Tailored solutions: Address specific business needs and optimize processes.

# Setup

In [7]:
import os

from dotenv import load_dotenv

from azure.identity import DefaultAzureCredential
from azure.ai.agents import AgentsClient
from azure.ai.agents.models import FunctionTool, ToolSet, ListSortOrder

load_dotenv()

AGENT_PROJECT_ENDPOINT = os.getenv("AGENT_PROJECT_ENDPOINT")
AGENT_MODEL_DEPLOYMENT = os.getenv("AGENT_MODEL_DEPLOYMENT")

agent_client = AgentsClient(
    endpoint=AGENT_PROJECT_ENDPOINT,
    credential=DefaultAzureCredential
        (exclude_environment_credential=True,
         exclude_managed_identity_credential=True)
)

for agent in agent_client.list_agents():
    print(agent.name)

snowfall-agent


# Agent with Function calling tool

In [1]:
import json
from typing import Set, Callable, Any

def recent_snowfall(location: str) -> str:
    """
    Fetches recent snowfall totals for a given location.
    :param location: The city name.
    :return: Snowfall details as a JSON string.
    """
    mock_snow_data = {"Seattle": "0 inches", "Denver": "2 inches"}
    snow = mock_snow_data.get(location, "Data not available.")
    return json.dumps({"location": location, "snowfall": snow})

user_functions: Set[Callable[..., Any]] = {
    recent_snowfall,
}

In [9]:
# Initialize agent toolset with user functions
functions = FunctionTool(user_functions)
toolset = ToolSet()
toolset.add(functions)
agent_client.enable_auto_function_calls(tools=toolset)

# Create your agent with the toolset
agent = agent_client.create_agent(
    model=AGENT_MODEL_DEPLOYMENT,
    name="snowfall-agent",
    instructions="You are a weather assistant tracking snowfall. Use the provided functions to answer questions.",
    toolset=toolset
)

In [10]:
agent.tools

[{'type': 'function', 'function': {'name': 'recent_snowfall', 'description': 'Fetches recent snowfall totals for a given location.', 'parameters': {'type': 'object', 'properties': {'location': {'type': 'string', 'description': 'The city name.'}}, 'required': ['location']}, 'strict': False}}]

In [11]:
# Create a thread for the conversation
thread = agent_client.threads.create()

In [12]:
# Send a prompt to the agent
message = agent_client.messages.create(
     thread_id=thread.id,
     role="user",
     content="What is the recent snowfall information in Denver?",
 )

run = agent_client.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)

# Get the conversation history
print("\nConversation Log:\n")
messages = agent_client.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)
for message in messages:
    if message.text_messages:
        last_msg = message.text_messages[-1]
        print(f"{message.role}: {last_msg.text.value}\n")


Conversation Log:

MessageRole.USER: What is the recent snowfall information in Denver?

MessageRole.AGENT: The recent snowfall in Denver is 2 inches.



In [13]:
# Clean up
agent_client.delete_agent(agent.id)